# Stencil Python Serial

Write the code to a file on disk:

In [5]:
%%writefile stc_pyt_seq.py
from time import time
t = time()
import numpy as np

n            = 4800    # nxn grid (4800,1,500)=1500
energy       = 1.0     # energy to be injected per iteration
niters       = 500     # number of iterations

size         = n + 2
sizeEnd      = n + 1
anew = aold  = np.zeros((size,  size), np.float64)
nsources     = 3       # sources of energy
sources      = np.empty((nsources, 2), np.int)
sources[:,:] = [ [n//2, n//2], [n//3, n//3], [n*4//5, n*8//9] ]
niters       = (niters+1) // 2

for iters in range(niters):
    anew[1:-1, 1:-1] =  ( aold[1:-1, 1:-1] / 2.0 + 
                        ( aold[2:  , 1:-1] + aold[ :-2, 1:-1] +
                          aold[1:-1, 2:  ] + aold[1:-1,  :-2] ) / 8.0 )
    anew[sources[0:nsources,0], sources[0:nsources,1]] += energy     
    aold[1:-1, 1:-1] =  ( anew[1:-1, 1:-1] / 2.0 +
                        ( anew[2:  , 1:-1] + anew[ :-2, 1:-1] +
                          anew[1:-1, 2:  ] + anew[1:-1,  :-2] ) / 8.0 )
    aold[sources[0:nsources,0], sources[0:nsources,1]] += energy   
heat = np.sum( aold[1:sizeEnd, 1:sizeEnd] )  # system total heat

t = time() - t
print("Heat = %0.4f | Tempo = %0.4f s" %(heat, t))

Overwriting stc_pyt_seq.py


Shell script to run Python code and also measure time using operating system time command:

In [1]:
%%writefile scriptshell.sh
#!/bin/sh
time python stc_pyt_seq.py

Writing scriptshell.sh


Check if it's running ok:

In [2]:
%%bash
sh scriptshell.sh

Heat = 1500.0000 | Tempo = 220.3673 s



real	3m45.647s
user	2m21.746s
sys	1m18.986s


Copy to the scratch area the files that will be executed in the execution nodes:

In [3]:
%%bash
s='/prj/ampemi/xxxx.xxxx/stnc/Python'
d='/scratch/ampemi/xxxx.xxxx/stnc/Python'
cp  $s/stc_pyt_seq.py $s/scriptshell.sh $d

Slurm batch script:

In [4]:
%%writefile stc_pyt_seq.srm
#!/bin/bash
#SBATCH -p cpu_small           # Select partition
#SBATCH --ntasks=1             # Total tasks(CPUs)
#SBATCH --nodes=1              # Number of nodes
#SBATCH --ntasks-per-node=1    # Number of tasks per node
#SBATCH -J stpyseq             # Job name
# #SBATCH --exclusive            # Exclusive acccess to nodes
#SBATCH --time=00:05:00        # Limit execution time

echo '========================================'
echo '- Job ID:' $SLURM_JOB_ID
echo '- Tasks per node:' $SLURM_NTASKS_PER_NODE
echo '- Number of nodes:' $SLURM_JOB_NUM_NODES
echo '- Total tasks:' $SLURM_NTASKS
echo '- Nodes alocated:' $SLURM_JOB_NODELIST
echo '- Directory where sbatch was called ($SLURM_SUBMIT_DIR):'
echo $SLURM_SUBMIT_DIR
cd $SLURM_SUBMIT_DIR

# Working dir
cd /scratch/ampemi/xxxx.xxxx/stnc/Python

# Executable
EXEC='sh scriptshell.sh'

# Run
echo '-- srun -------------------------------'
echo '$ srun -n ' $SLURM_NTASKS $EXEC
srun -n $SLURM_NTASKS $EXEC
echo '-- END --------------------------------'

Overwriting stc_pyt_seq.srm


<hr style="height:10px;border-width:0;background-color:green">

Submit the batch script to Slurm, 3 times:

In [5]:
%%bash
sbatch stc_pyt_seq.srm
sbatch stc_pyt_seq.srm
sbatch stc_pyt_seq.srm
squeue -n stpyseq

Submitted batch job 772572
Submitted batch job 772573
Submitted batch job 772574
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
            772572 cpu_small  stpyseq xxxx. PD       0:00      1 (Priority)
            772573 cpu_small  stpyseq xxxx. PD       0:00      1 (Priority)
            772574 cpu_small  stpyseq xxxx. PD       0:00      1 (Priority)


Check if it's finished:

Shows the files containing the outputs:

In [8]:
%%bash
b='/stnc/Python'
d='/scratch/ampemi/xxxx.xxxx/'$b
cat $d/slurm-772572.out
cat $d/slurm-772573.out
cat $d/slurm-772574.out

- Job ID: 772572
- Tasks per node: 1
- Number of nodes: 1
- Total tasks: 1
- Nodes alocated: sdumont1405
- Directory where sbatch was called ($SLURM_SUBMIT_DIR):
/prj/ampemi/xxxx.xxxx/stnc/Python
-- srun -------------------------------
$ srun -n  1 sh scriptshell.sh
Heat = 1500.0000 | Tempo = 225.1292 s

real	3m46.153s
user	2m56.529s
sys	0m48.873s
-- END --------------------------------
- Job ID: 772573
- Tasks per node: 1
- Number of nodes: 1
- Total tasks: 1
- Nodes alocated: sdumont1306
- Directory where sbatch was called ($SLURM_SUBMIT_DIR):
/prj/ampemi/xxxx.xxxx/stnc/Python
-- srun -------------------------------
$ srun -n  1 sh scriptshell.sh
Heat = 1500.0000 | Tempo = 187.1582 s

real	3m7.998s
user	2m24.770s
sys	0m42.647s
-- END --------------------------------
- Job ID: 772574
- Tasks per node: 1
- Number of nodes: 1
- Total tasks: 1
- Nodes alocated: sdumont1405
- Directory where sbatch was called ($SLURM_SUBMIT_DIR):
/prj/ampemi/xxxx.xxxx/stnc/Python
-- srun -----------------

<hr style="height:10px;border-width:0;background-color:red">

## Version

In [1]:
import sys
print (sys.version)
print (sys.version_info)

3.7.3 | packaged by conda-forge | (default, Jul  1 2019, 21:52:21) 
[GCC 7.3.0]
sys.version_info(major=3, minor=7, micro=3, releaselevel='final', serial=0)


In [2]:
import numpy
numpy.show_config()

blas_mkl_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/scratch/app/anaconda3/2018.12/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/scratch/app/anaconda3/2018.12/include']
blas_opt_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/scratch/app/anaconda3/2018.12/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/scratch/app/anaconda3/2018.12/include']
lapack_mkl_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/scratch/app/anaconda3/2018.12/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/scratch/app/anaconda3/2018.12/include']
lapack_opt_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/scratch/app/anaconda3/2018.12/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/scratch/app/anaconda3/2018.12/include']
